# Capítulo 6 - Tarefas Continuadas - Differential Q-Learning


Você pode rodar este notebook localmente ou no Colab. Para abrir diretamente no Colab, basta clicar no link abaixo.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pablo-sampaio/rl_facil/blob/main/cap06/cap06-main-1.ipynb)

## Configurações Iniciais

In [ ]:
from IPython.display import clear_output
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install gym==0.23.1
    !pip install optuna

    # clone repository
    !git clone https://github.com/pablo-sampaio/rl_facil
    sys.path.append("/content/rl_facil")

    clear_output()
else:
    from os import path
    sys.path.append( path.dirname( path.dirname( path.abspath("__main__") ) ) )

clear_output()

In [ ]:
import gym
from gym import spaces
import numpy as np
import optuna

from util.experiments import repeated_exec, repeated_exec_greedy_Q
from util.plot import plot_result, plot_multiple_results

## 1 - Tarefa Continuada (Infinita)

In [ ]:
class TwoChoice(gym.Env):
    def __init__(self):
        super().__init__()

        # Define the action and observation spaces
        self.action_space = spaces.Discrete(2)  # Two discrete actions: 0 and 1
        self.observation_space = spaces.Discrete(9)  # Nine discrete states: 0 to 8

        # Set the initial state
        self.current_state = 0

    def reset(self):
        # Reset the environment to the initial state
        self.current_state = 0
        return self.current_state

    def step(self, action):
        # Perform the specified action and transition to the next state
        if action != 0 and action != 1:
            raise ValueError("Invalid action!")
        
        reward = 0.0

        if self.current_state == 0:
            # left
            if action == 0:
                reward = 1.0
                self.current_state = 1
            # right
            else:
                reward = 0.0
                self.current_state = 5
        elif self.current_state == 4:
            reward = 0.0
            self.current_state = 0
        elif self.current_state == 8:
            reward = 2.0
            self.current_state = 0
        else:
            reward = 0.0
            self.current_state += 1

        return self.current_state, reward, False, {}

    def render(self, mode=None):
        # Display the current state (optional)
        print("Current state:", self.current_state)


## 2 - Q-Learning (parando por passos)

O mesmo **Q-learning** que vimos antes, porém com um critério de parada dado como uma quantidade de passos (não importa a quantidade de episódios envolvida).

In [ ]:
# Esta é a política. Neste caso, escolhe uma ação com base nos valores
# da tabela Q, usando uma estratégia epsilon-greedy.
def epsilon_greedy(Q, state, num_actions, epsilon):
    if np.random.random() < epsilon:
        return np.random.randint(0, num_actions)
    else:
        return np.argmax(Q[state])

In [ ]:
# Algoritmo Q-learning
def run_qlearning_step(env, total_steps, lr=0.1, gamma=0.95, epsilon=0.1):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)

    num_actions = env.action_space.n
    
    # inicializa a tabela Q com valores aleatórios de -1.0 a 0.0
    # usar o estado como índice das linhas e a ação como índice das colunas
    Q = np.random.uniform(low = -1.0, high = 0.0, 
                          size = (env.observation_space.n, num_actions))

    # para cada episódio, guarda sua soma de recompensas (retorno não-descontado)
    sum_rewards_per_step = []

    state = env.reset()
    sum_rewards, reward = 0, 0

    # loop principal
    for i in range(total_steps):
        
        # escolhe a próxima ação -- usa epsilon-greedy
        action = epsilon_greedy(Q, state, num_actions, epsilon)
    
        # realiza a ação, ou seja, dá um passo no ambiente
        next_state, reward, done, _ = env.step(action)
        
        if done: 
            # para estados terminais
            V_next_state = 0
            next_state = env.reset()
        else:
            # para estados não-terminais -- valor máximo (melhor ação)
            V_next_state = np.max(Q[next_state])

        # atualiza a Q-table
        # delta = (estimativa usando a nova recompensa) - estimativa antiga
        delta = (reward + gamma * V_next_state) - Q[state,action]
        Q[state,action] = Q[state,action] + lr * delta
        
        sum_rewards += reward
        sum_rewards_per_step.append(sum_rewards)

        state = next_state

        # a cada 1000 passos, imprime informação sobre o progresso 
        if (i+1) % 1000 == 0:
            avg_reward = np.mean(sum_rewards_per_step[-100:])
            print(f"Step {i+1} Average Reward (last 100): {avg_reward:.3f}")

    return sum_rewards_per_step, Q

In [ ]:
TOTAL_STEPS = 100
LR = 0.3
GAMMA = 0.70   # só vai dar a política ótima para valores a partir de 0.85
EPSILON = 0.1

rmax = 2*TOTAL_STEPS
env = TwoChoice()

rewards, qtable = run_qlearning_step(env, TOTAL_STEPS, LR, GAMMA, EPSILON)
print("Acumulado final =", (rewards[-1]))

In [ ]:
# Mostra um gráfico de episódios x retornos não descontados
plot_result(rewards, rmax, None, window=1)

## 3 - Differential Q-Learning

Um algoritmo específico para tarefas continuadas.

In [ ]:
# Algoritmo Differential Q-learning
def run_differential_qlearning_step(env, total_steps, lr=0.1, eta=1.0, epsilon=0.1):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)

    num_actions = env.action_space.n
    
    # inicializa a tabela Q com valores aleatórios de -1.0 a 0.0
    # usar o estado como índice das linhas e a ação como índice das colunas
    Q = np.random.uniform(low = -1.0, high = 0.0, 
                          size = (env.observation_space.n, num_actions))

    # para cada episódio, guarda sua soma de recompensas (retorno não-descontado)
    sum_rewards_per_step = []

    state = env.reset()
    sum_rewards, reward = 0, 0
    mean_reward = 0.0

    # loop principal
    for i in range(total_steps):
        
        # escolhe a próxima ação -- usa epsilon-greedy
        action = epsilon_greedy(Q, state, num_actions, epsilon)
    
        # realiza a ação, ou seja, dá um passo no ambiente
        next_state, reward, done, _ = env.step(action)
        
        assert not done, "This algorithm is for continuing tasks!"

        # para estados não-terminais -- valor máximo (melhor ação)
        V_next_state = np.max(Q[next_state])

        # atualiza a Q-table
        # delta = (estimativa usando a nova recompensa) - estimativa antiga
        delta = (reward - mean_reward + V_next_state) - Q[state,action]
        Q[state,action] = Q[state,action] + lr * delta

        # atualiza a recompensa média
        mean_reward += eta * lr * delta
        
        sum_rewards += reward
        sum_rewards_per_step.append(sum_rewards)

        state = next_state

        # a cada 1000 passos, imprime informação sobre o progresso 
        if (i+1) % 1000 == 0:
            avg_reward = np.mean(sum_rewards_per_step[-100:])
            print(f"Step {i+1} Average Reward (last 100): {avg_reward:.3f}")

    return sum_rewards_per_step, Q

In [ ]:
TOTAL_STEPS = 100
LR = 0.3
ETA = 1.0
EPSILON = 0.1

rmax = 2*TOTAL_STEPS
env = TwoChoice()

rewards, qtable = run_differential_qlearning_step(env, TOTAL_STEPS, LR, ETA, EPSILON)
print("Acumulado final =", (rewards[-1]))

In [ ]:
# Mostra um gráfico de episódios x retornos não descontados
plot_result(rewards, rmax, None, window=1)

## 4 - Otimizando Parâmetros

Vamos usar a biblioteca *Optuna* para otimizar (hiper-)parâmetros dos algoritmos de treinamento.

In [ ]:
ENV   = TwoChoice()
RUNS_PER_TRIAL = 10

### 4.1 - Q-Learning

In [ ]:
def create_train_fn(fixed_gamma):
    def train(trial : optuna.Trial):
        # chama os métodos do "trial" (tentativa) para sugerir valores para os parâmetros
        lr = trial.suggest_uniform('lr', 0.1, 1.0)
        eps = trial.suggest_uniform('epsilon', 0.01, 0.2)

        print(f"\nTRIAL #{trial.number}: lr={lr}, eps={eps}")

        # roda o algoritmo várias vezes
        results = repeated_exec(RUNS_PER_TRIAL, "qlearn-optuna", run_qlearning_step, ENV, 100, lr=lr, epsilon=eps, gamma=fixed_gamma)
        
        # soma dos retornos não-descontado finais
        return np.sum(results[1][:,-1])
    return train

In [ ]:
GAMMA = 0.70
study = optuna.create_study(direction='maximize',
                        storage='sqlite:///optuna_continuing.db',
                        study_name=f'qlearning-g{GAMMA}',
                        load_if_exists=True)

study.optimize(create_train_fn(GAMMA), n_trials=100)
clear_output()

print("MELHORES PARÂMETROS PARA GAMMA", GAMMA, ":")
print(study.best_params)
qlearn_params_g07 = study.best_params
qlearn_params_g07['gamma'] = GAMMA

In [ ]:
GAMMA = 0.95
study = optuna.create_study(direction='maximize',
                        storage='sqlite:///optuna_continuing.db',
                        study_name=f'qlearning-g{GAMMA}',
                        load_if_exists=True)

study.optimize(create_train_fn(GAMMA), n_trials=100)
clear_output()

print("MELHORES PARÂMETROS PARA GAMMA", GAMMA, ":")
print(study.best_params)
qlearn_params_g09 = study.best_params
qlearn_params_g09['gamma'] = GAMMA

### 4.2 - Differential Q-Learning

In [ ]:
def train_diff(trial : optuna.Trial):
    # chama os métodos do "trial" (tentativa) para sugerir valores para os parâmetros
    lr = trial.suggest_uniform('lr', 0.1, 1.0)
    eps = trial.suggest_uniform('epsilon', 0.01, 0.20)
    eta = trial.suggest_uniform('eta', 0.1, 2.0)

    print(f"\nTRIAL #{trial.number}: lr={lr}, eps={eps}, eta={eta}")

    # roda o algoritmo várias vezes
    results = repeated_exec(RUNS_PER_TRIAL, "diff-q-optuna", run_differential_qlearning_step, ENV, 100, lr=lr, epsilon=eps, eta=eta)
    
    # soma dos retornos não-descontado finais
    return np.sum(results[1][:,-1])


In [ ]:
study = optuna.create_study(direction='maximize',
                        storage='sqlite:///optuna_continuing.db',
                        study_name='diff-qlearning',
                        load_if_exists=True)

study.optimize(train_diff, n_trials=100)
clear_output()

print("MELHORES PARÂMETROS:")
print(study.best_params)
diff_qlearn_params = study.best_params

## 5 - Experimentos

### 5.1 - Desempenho no Treinamento

Comparando os dois usando os parâmetros ótimos obtidos antes.

In [ ]:
environment = TwoChoice()
NUM_STEPS = 200
RUNS = 200

results = []

results.append( repeated_exec(RUNS, f"Q-Learning (g=0.7)", run_qlearning_step, environment, NUM_STEPS, **qlearn_params_g07) )
clear_output()

results.append( repeated_exec(RUNS, f"Q-Learning (g=0.9)", run_qlearning_step, environment, NUM_STEPS, **qlearn_params_g09) )
clear_output()

results.append( repeated_exec(RUNS, f"Diff Q-Learning", run_differential_qlearning_step, environment, NUM_STEPS, **diff_qlearn_params) )
clear_output()

In [ ]:
plot_multiple_results(results, cumulative=False, x_log_scale=False, window=1)

In [ ]:
plot_multiple_results(results[0:1], cumulative=False, x_log_scale=False, window=1, plot_stddev=True)

In [ ]:
plot_multiple_results(results[1:2], cumulative=False, x_log_scale=False, window=1, plot_stddev=True)

In [ ]:
plot_multiple_results(results[2:], cumulative=False, x_log_scale=False, window=1, plot_stddev=True)

### 5.2 - Desempenho Pós-Treinamento

In [ ]:
environment = TwoChoice()
NUM_STEPS = 1000

In [ ]:
_, qtable1 = run_qlearning_step(environment, NUM_STEPS, **qlearn_params_g07)
#_, qtable2 = run_qlearning_step(environment, NUM_STEPS, **qlearn_params_g09)
_, qtable3 = run_differential_qlearning_step(environment, NUM_STEPS, **diff_qlearn_params)

In [ ]:
results = []

results.append( repeated_exec_greedy_Q(1, "QLearn(0.7)-Greedy", qtable1, environment, 1000) )
#results.append( repeated_exec_greedy_Q(1, "QLearn(0.9)-Greedy", qtable2, environment, 1000) )
results.append( repeated_exec_greedy_Q(1, "Diff-QLearn-Greedy", qtable3, environment, 1000) )

In [ ]:
plot_multiple_results(results, cumulative=False, x_log_scale=False, window=1)